In [20]:
import numpy as np
import matplotlib.pyplot as plt

In [21]:
def res_sim(inflow,evap,demand,s_0,s_max,env_min,supply):
    
    """
    This is a model that simulates the operation of a single reservoir system. 
    It essentially consists of a water balance equation, 
    where the storage (s) at a future time step is predicted from the storage at the current time 
    by adding and subtracting the inflows and outflows that will occur during the temporal interval ahead

    The inputs of the model are:

    inflow = time series of reservoir inflows [ML]
    evap = time series of evaporation from the reservoir surface area [ML]
    demand = time series of water demand [ML]
    s_0 = initial reservoir storage [ML]
    s_max = maximum storage capacity of the reservoir [ML]
    env_min = minimum environmental flow [ML]
    supply = regulated reservoir release for water supply [ML]
    
    And the outpus are:
    
    s = reservoir storage [ML]
    env = environmental compensation flow [ML]
    spill = outflow through spillways [ML]
    supply = regulated reservoir release for water supply [ML]
    
    """
    T = len(inflow) # number of time steps (weeks)
    # Declare output variables

    s = np.zeros(T+1) # reservoir storage in ML

    spill = np.zeros(T) # spillage in ML

    env = np.zeros(T) + env_min # environmental compensation flow
    
    # Initial storage
    s[0] = s_0
    
    for t in np.arange(T):
        # Environmental flow
        if s[t]+inflow[t]-evap[t] <= 0:
            env[t] = 0
            
        if env[t] >= s[t]+inflow[t]-evap[t]:
            env[t] = s[t]+inflow[t]-evap[t]
            
        # Supply
        if s[t]+inflow[t]-evap[t]-env[t] <= 0:
            supply[t] = 0
            
        if supply[t] >= s[t]+inflow[t]-evap[t]-env[t]:
            supply[t] = s[t]+inflow[t]-evap[t]-env[t]
            
        # Spillage
        if s[t]+inflow[t]-evap[t]-env[t]-supply[t] > s_max:
            spill[t] = 𝑠[t]+inflow[t]-evap[t]-env[t]-supply[t] - s_max
        
        s[t+1]=s[t]+inflow[t]-evap[t]-env[t]-spill[t]-supply[t]
        
    return s,env,spill,supply

In [22]:
# Define the MSV function
def MSV_func(s, s_min):
    msv = np.abs(np.maximum(s_min - s, 0))
    return np.mean(msv)

# Define the SD function
def SD_func(supply, demand):
    sd = np.maximum(demand - supply, 0) ** 2
    return np.mean(sd)

In [23]:
def fit_func(pop_size,population):
    objectives = np.zeros((pop_size, 2))
    for i in range(pop_size):
        supply = population[i]*demand
        s, env, spill, supply = res_sim(inflow, evap, demand, s_0, s_max, env_min, supply)
        objectives[i, 0] = MSV_func(s[:-1], s_min)
        objectives[i, 1] = SD_func(supply, demand)
    
    return objectives

In [24]:
# Define dominates function
def dominates(obj1, obj2):
    return np.all(obj1 <= obj2) and np.any(obj1 < obj2)

In [25]:
def parents_selection(objectives,population):
    domination_count = np.zeros(len(objectives))
    dominated_solutions = [[] for _ in range(len(objectives))]
    for i in range(len(objectives)):
        for j in range(i + 1, len(objectives)):
            if dominates(objectives[i], objectives[j]):
                dominated_solutions[i].append(j)
                domination_count[j] += 1
            elif dominates(objectives[j], objectives[i]):
                dominated_solutions[j].append(i)
                domination_count[i] += 1
    print(domination_count)
    parent_1 = population[np.where(domination_count == 0)[0][0]]
    parent_2 = population[np.where(domination_count == 1)[0][0]]
    
    return parent_1,parent_2

In [26]:
def crossover(parent_1, parent_2):
    
    num_genes = len(parent_1)
    crossover_ratios = np.arange(0.25,1,0.25)
    new_population = np.row_stack((parent_1,parent_2))
    for crossover_ratio in crossover_ratios:
        crossover_point = int(num_genes * crossover_ratio)
        child_1 = np.append(parent_1[:crossover_point],parent_2[crossover_point:])
        child_2 = np.append(parent_2[:crossover_point],parent_1[crossover_point:])
        new_population=np.row_stack((new_population,child_1,child_2))
        
    return new_population

In [30]:
def gen_opt(max_generations):
    T = len(inflow)
    num_weeks = len(inflow)
    # Initialize population
    population = np.random.uniform(0, 1, (pop_size, num_weeks))
    
    # Initialize best solutions
    best_solutions = []
    best_objectives = []
    # Iterate through generations
    for gen in range(max_generations):
        # Evaluate population
        objectives = fit_func(pop_size,population)
        parent_1, parent_2 = parents_selection(objectives,population)
        population = crossover(parent_1, parent_2)
        print(population)
    return population, objectives


In [31]:
# Define inputs
inflow = np.array([15, 17, 19, 11, 9, 4, 3, 8])  # (ML/week) time series of inflow forecasts
evap = np.array([1, 1, 2, 2, 2, 2, 2, 3])  # (ML/week) time series of evaporation forecasts
demand = np.array([13, 13, 17, 18, 20, 22, 25, 26])  # (ML/week) time series of demand forecasts
T = len(inflow)  # number of time steps (weeks)
s_max = 150  # (ML) Maximum storage (=reservoir capacity)
s_min = 30  # (ML) Minimum storage
env_min = 2  # (ML/week) Environmental compensation flow
s_0 = 80  # (ML) Storage volume at the beginning of the simulation period

# Perform multi-objective optimization
pop_size = 8
max_generations = 10

In [32]:
best_solutions, best_objectives = gen_opt(max_generations)

# Plot the Pareto front
plt.scatter(best_objectives[:, 0], best_objectives[:, 1])
plt.xlabel('Mean Storage Violations (MSV)')
plt.ylabel('Mean Squared Deviation (SD)')
plt.title('Pareto Front')
plt.show()

[6. 5. 0. 7. 4. 1. 2. 3.]
[[0.62768224 0.94134916 0.71410188 0.83466873 0.23522822 0.70494656
  0.71643841 0.59569453]
 [0.7618303  0.26205766 0.21524913 0.651884   0.30885083 0.36083368
  0.92206725 0.63851129]
 [0.62768224 0.94134916 0.21524913 0.651884   0.30885083 0.36083368
  0.92206725 0.63851129]
 [0.7618303  0.26205766 0.71410188 0.83466873 0.23522822 0.70494656
  0.71643841 0.59569453]
 [0.62768224 0.94134916 0.71410188 0.83466873 0.30885083 0.36083368
  0.92206725 0.63851129]
 [0.7618303  0.26205766 0.21524913 0.651884   0.23522822 0.70494656
  0.71643841 0.59569453]
 [0.62768224 0.94134916 0.71410188 0.83466873 0.23522822 0.70494656
  0.92206725 0.63851129]
 [0.7618303  0.26205766 0.21524913 0.651884   0.30885083 0.36083368
  0.71643841 0.59569453]]
[1. 6. 4. 3. 2. 5. 0. 7.]
[[0.62768224 0.94134916 0.71410188 0.83466873 0.23522822 0.70494656
  0.92206725 0.63851129]
 [0.62768224 0.94134916 0.71410188 0.83466873 0.23522822 0.70494656
  0.71643841 0.59569453]
 [0.62768224 0.94

IndexError: index 0 is out of bounds for axis 0 with size 0